In [2]:
from pprint import pprint
import time
import traceback
from random import randint
from itertools import cycle
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [3]:
def get_driver(proxy=None, user_agent=None):
    options = Options()
    options.headless = True

    if proxy:
        options.add_argument(f'--proxy-server={proxy}')
    if user_agent:
        options.add_argument(f'--user_agent={user_agent}')

    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference('permissions.default.image', 2)
    firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
    browser = webdriver.Firefox(options=options, firefox_profile=firefox_profile)

    return browser

In [5]:
def parsed_url(page = 1, low_range=250, up_range=5000, build_link = None):
    base_url = 'https://pcpartpicker.com'
    if build_link == None:
        fragment = f'/builds/#B=1&page={page}&X={low_range}00,{up_range}00'
    else: 
        fragment = f'{build_link}'

    return f'{base_url}{fragment}'

In [123]:
def clean_price(price):
    if price[0] == '$':
        price = price.replace('$', '').strip()
        if len(price.split(' ')) > 1:
            return False
    else: 
        return False

    return float(price)

In [158]:
def build_scraper(url, user_agent):
    builds_dict = {}
    build_comps = ['Name','CPU', 'CPU Cooler', 'Motherboard', 'Memory', 'Storage', 'Video Card', 'Case', 'Power Supply', 'Build Price']

    try:
        rq = requests.get(url, headers=user_agent)
    except Exception as e:
        print(e)
        return builds_dict

    soup = BeautifulSoup(rq.content, 'lxml')
    builds_dict['Name'] = soup.find('h1', {"class": "build__name"}).text
    comp_table_rows = soup.find('table', {"class": "partlist partlist--mini"}).find_all('tr')
    extra_price = 0

    # Two rows is one component, one for the name of the comp and other for the features
    row_it = iter(comp_table_rows)
    for name, component in zip(row_it, row_it):
        try:
            name_text = name.find('h4').text.strip()
            # Getting the name and price components
            component_el = component.find('td', {'class':'td__name'}).findChildren(text=True)
            component_el = list(filter(lambda el: el != '\n', component_el))

            if len(component_el) == 2:
                comp_name = component_el[0]
                comp_price = clean_price(component_el[1])
                # If price isn't in USD
                if not comp_price: 
                    return {}
            else:
                comp_name, comp_price = *component_el, None

            # If the component are in the selected list for scrape
            if name_text in build_comps:
                comp_els = {'Name': comp_name, 'Price': comp_price}

                if name_text not in builds_dict:
                    builds_dict[name_text] = comp_els
                else:
                    comp_copy = builds_dict[name_text].copy()
                    builds_dict[name_text] = []
                    builds_dict[name_text].extend([comp_els, comp_copy])
            else:
                # Calculate the total of the components not taken into account
                extra_price += comp_price if isinstance(comp_price, float) else 0

            
        except Exception as e:
            print(e, url, name_text, component_el)
            continue

        total_table_row = soup.find('table', {"class": "block partlist partlist--mini partlist--totals"}).find('td', {"class": "td__price"}).text
        builds_dict['Build Price'] = round(float(total_table_row.replace('$', '')) - extra_price, 2)
    
    return builds_dict

In [159]:
def main():
    user_agent = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    n_pages = 1
    builds_list = [] 
    for i in range(1, n_pages+1):
        try:
            url = parsed_url(page=i)
            browser = get_driver(user_agent=user_agent)
            browser.get(url)
            delay = randint(2, 5)  
            time.sleep(delay)
            soup = BeautifulSoup(browser.page_source, 'lxml')

            # Get the link of all build cards in a single page
            builds_links = soup.find_all("a", {"class": "logGroup__target"}, href=True)
            
            for build in builds_links:
                build_url = parsed_url(build_link=build['href'])
                build_dict = build_scraper(build_url, user_agent)
                if build_dict:
                    builds_list.append(build_dict)
                
                delay = randint(2, 10)  
                time.sleep(delay)

            browser.close()
        except Exception as e:
            browser.close()
            print(traceback.format_exc())
            continue

    
    return builds_list


In [156]:
if __name__ == '__main__':
    builds = build_scraper('https://pcpartpicker.com/b/Bstp99', user_agent)

In [157]:
pprint(builds)

{'Build Price': 1421.35,
 'CPU': {'Name': 'Intel Core i7-9700K 3.6 GHz 8-Core', 'Price': 289.99},
 'CPU Cooler': {'Name': 'Corsair iCUE H100i RGB PRO XT 75 CFM Liquid',
                'Price': 119.99},
 'Case': {'Name': 'RIOTORO CR1080 ATX Mid Tower', 'Price': 132.4},
 'Memory': [{'Name': 'Corsair Vengeance LPX 32 GB (2 x 16 GB) DDR4-3200 CL16',
             'Price': 129.99},
            {'Name': 'Corsair Vengeance LPX 32 GB (2 x 16 GB) DDR4-3200 CL16',
             'Price': 129.99}],
 'Motherboard': {'Name': 'EVGA Z390 FTW ATX LGA1151', 'Price': None},
 'Name': 'Tiny HTPC Gaming Rig with CR1080',
 'Power Supply': {'Name': 'Corsair 860 W 80+ Platinum Certified Fully Modular '
                          'ATX',
                  'Price': None},
 'Storage': [{'Name': 'Hitachi Ultrastar 7K4000 3 TB 3.5" 7200RPM',
              'Price': None},
             {'Name': 'Samsung 970 Evo 500 GB M.2-2280 NVME SSD',
              'Price': 89.0}],
 'Video Card': {'Name': 'Asus GeForce RTX 2070 SUPER